<a href="https://colab.research.google.com/github/zakcroft/fine-tuning-notebooks/blob/main/fine_tuning_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets config transformers[torch] lamini evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/deeplearning_fine_tuning

/content/drive/My Drive/Colab Notebooks/deeplearning_fine_tuning


In [ ]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import logging
import time
import torch
import transformers
import numpy as np
import evaluate

# from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import Trainer

In [ ]:
!ls
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)


Fine_tuning_data_prepare.ipynb	lamini_docs_3_steps  __pycache__
fine_tuning_training.ipynb	lamini_docs.jsonl    utilities.py


In [ ]:
device_count = torch.cuda.device_count()
logger.debug("Checking device")
if device_count > 0:
    print("Select GPU device")
    device = torch.device("cuda")
else:
    print("Select CPU device")
    device = torch.device("cpu")

Select GPU device


In [ ]:
model_name = "EleutherAI/pythia-70m"
base_model = AutoModelForCausalLM.from_pretrained(model_name)
base_model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
dataset_path = "lamini/lamini_docs"

dataset = datasets.load_dataset(dataset_path)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(train_dataset[0]['question'])
print(train_dataset[0]['answer'])

print(test_dataset[0]['question'])
print(test_dataset[0]['answer'])

How can I evaluate the performance and quality of the generated text from Lamini models?
There are several metrics that can be used to evaluate the performance and quality of generated text from Lamini models, including perplexity, BLEU score, and human evaluation. Perplexity measures how well the model predicts the next word in a sequence, while BLEU score measures the similarity between the generated text and a reference text. Human evaluation involves having human judges rate the quality of the generated text based on factors such as coherence, fluency, and relevance. It is recommended to use a combination of these metrics for a comprehensive evaluation of the model's performance.
Can Lamini generate technical documentation or user manuals for software projects?
Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical 

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["question"], padding=True, truncation=True,  return_tensors="pt", max_length=512)

small_train_dataset = dataset["train"].select(range(1))
small_test_dataset = dataset["test"].select(range(1))

# print(train_dataset)
print(small_train_dataset[0])
print(small_test_dataset[0])

# encode
encoding_dataset = small_test_dataset.map(tokenize_function, batched=True)
input_ids=torch.tensor(encoding_dataset['input_ids']).to(device)
attention_mask = torch.tensor(encoding_dataset['attention_mask']).to(device)

# ask
base_model_generated_tokens_with_prompt = base_model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=512
  )

# decode
generated_text_with_prompt = tokenizer.batch_decode(base_model_generated_tokens_with_prompt, skip_special_tokens=True)
print(generated_text_with_prompt)

{'question': 'How can I evaluate the performance and quality of the generated text from Lamini models?', 'answer': "There are several metrics that can be used to evaluate the performance and quality of generated text from Lamini models, including perplexity, BLEU score, and human evaluation. Perplexity measures how well the model predicts the next word in a sequence, while BLEU score measures the similarity between the generated text and a reference text. Human evaluation involves having human judges rate the quality of the generated text based on factors such as coherence, fluency, and relevance. It is recommended to use a combination of these metrics for a comprehensive evaluation of the model's performance.", 'input_ids': [2347, 476, 309, 7472, 253, 3045, 285, 3290, 273, 253, 4561, 2505, 432, 418, 4988, 74, 3210, 32, 2512, 403, 2067, 17082, 326, 476, 320, 908, 281, 7472, 253, 3045, 285, 3290, 273, 4561, 2505, 432, 418, 4988, 74, 3210, 13, 1690, 44229, 414, 13, 378, 1843, 54, 4868, 1

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Can Lamini generate technical documentation or user manuals for software projects?\n\nI have a question about the following:\n\nHow do I get the correct documentation to work?\n\nA:\n\nI think you need to use the following code:\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the foll

In [ ]:
# print("Question input:", small_train_dataset[0]['question'])
# print("Correct answer from Lamini docs:", small_train_dataset[0]['answer'])

# # Strip the prompt
base_model_generated_answer = generated_text_with_prompt[0][len(small_test_dataset[0]['question']):].replace('.', '.\n')

print('Models answer:', base_model_generated_answer)

Models answer: 

I have a question about the following:

How do I get the correct documentation to work?

A:

I think you need to use the following code:

A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentat

In [ ]:

# Now train

max_steps = -1
epochs=2
batch_size=1

trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=epochs,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=batch_size,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, 2048)
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

# print(base_model)
# print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
# print("Flops", model_flops / 1e9, "GFLOPs")

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from torch.utils.data import Dataset
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # Assumes `tokenizer` is previously defined


class FilteredDataset(Dataset):
    def __init__(self, original_dataset):
        self.filtered_data = [item for item in original_dataset if len(item['input_ids']) > 0]

    def __getitem__(self, idx):
        return self.filtered_data[idx]

    def __len__(self):
        return len(self.filtered_data)
small_train_dataset = dataset["train"].select(range(230))
small_test_dataset = dataset["train"].select(range(230))
filtered_train_dataset = FilteredDataset(small_train_dataset)
filtered_test_dataset = FilteredDataset(small_test_dataset)

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=filtered_train_dataset,
    eval_dataset=filtered_test_dataset,
    data_collator=data_collator,  # Add this line
    compute_metrics=compute_metrics,
)


In [ ]:
training_output = trainer.train()

print(training_output)

Step,Training Loss,Validation Loss


TrainOutput(global_step=114, training_loss=0.517733567639401, metrics={'train_runtime': 15.3871, 'train_samples_per_second': 29.895, 'train_steps_per_second': 7.409, 'total_flos': 9723173978112.0, 'train_loss': 0.517733567639401, 'epoch': 1.98})


In [ ]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

finetuned_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)

finetuned_model_output = finetuned_model.to(device)


Saved model to: lamini_docs_-1_steps/final


In [ ]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(10))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(10))

# encode
encoding_dataset = small_train_dataset.map(tokenize_function, batched=True)
input_ids=torch.tensor(encoding_dataset['input_ids']).to(device)
attention_mask = torch.tensor(encoding_dataset['attention_mask']).to(device)

# ask
finetuned_generated_tokens_with_prompt = finetuned_model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=512
  )

print(finetuned_generated_tokens_with_prompt)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


tensor([[ 2513,   352,  1896,  ..., 11793, 29600, 28606],
        [ 2347,  2608,    14,  ...,  5859,   941,   941],
        [ 5804,   253, 10097,  ...,   715,   253,  4795],
        ...,
        [10795,   418,  4988,  ...,  2278,   285, 12921],
        [10795,   418,  4988,  ..., 28282,    84,    15],
        [ 2347,   513,   309,  ..., 19007,   273, 10414]], device='cuda:0')


In [ ]:
# decode
finetuned_generated_text_with_prompt = tokenizer.batch_decode(finetuned_generated_tokens_with_prompt, skip_special_tokens=True)


print("Question input:", small_train_dataset[0]['question'])
print("Correct answer from Lamini docs:", small_train_dataset[0]['answer'])
print(finetuned_generated_text_with_prompt)
# # Strip the prompt
finetuned_generated_answer = finetuned_generated_text_with_prompt[0][len(small_train_dataset[0]['question']):].replace('?', '?\n')

print('Models answer:', finetuned_generated_answer)

Question input: Is it possible to use Lamini for generating poetry or creative writing?
Correct answer from Lamini docs: Yes, it is possible to use Lamini for generating poetry or creative writing. The LLM Engine can be trained on a dataset of poems or creative writing, and then used to generate new pieces based on that training. Additionally, the LLM Engine can be fine-tuned on a specific style or genre of poetry or creative writing to generate more targeted results.
['Is it possible to use Lamini for generating poetry or creative writing?Lamini can be used to generate poetry or write for free. It can be fine-tuned or customized to suit your specific needs. Try it out for yourself or your organization. Feedback on your writing prowess is greatly appreciated. Feedback on success stories is appreciated. Feedback on losing your writing ability is greatly appreciated. Feedback on losing your writing ability is greatly appreciated. Feedback on success stories is appreciated. Feedback on lo

In [ ]:
# Strip the prompt
finetuned_generated_text_answer = finetuned_generated_text_with_prompt[0][len(small_train_dataset[0]['question']):]

base_model_modified_text = base_model_generated_answer.replace("?", "?\n")
finetuned_modified_text = finetuned_generated_text_answer.replace("?", "?\n")

print(base_model_modified_text)
print('===============')
print(finetuned_modified_text)



I have a question about the following:

How do I get the correct documentation to work?


A:

I think you need to use the following code:

A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You can use the following code to get the correct documentation.


A:

You